In [ ]:
import pickle
import numpy as np
import distance as dist
from collections import Counter
import re

In [ ]:
# 100 clusters of 100 words to reduce the computational complexity
kmeans_clusters = pickle.load(open("kmeans_word_clusters.data", 'rb'))
# we use only the 10,000 most common words to reduce computational complexity
words_dictionary = pickle.load(open("words.data", 'rb'))
# number of occurences of our words
word_counter = Counter(re.findall(r'\w+', open('big.txt').read().lower()))

In [183]:
def get_corrections(word):
    closest_clusters = find_closest_clusters(word)
#     print('closest_clusters:')
#     print(closest_clusters)
    closest_words = find_closest_words(word, closest_clusters)
#     print('closest_words:')
#     print(closest_words)
    suggestions = get_suggestions(closest_words)
    
    for word in suggestions:
        print(word)

# find clusters with the lowest distance to the word
def find_closest_clusters(word):
    # max distance to the word representing the cluster
    MAX_DISTANCE = 4
    cluster_distances = []
    for i in range(len(kmeans_clusters)):
        # check distance to one of the words in the cluster
        cluster_distances.append([i, dist.levenshtein(word, kmeans_clusters[i][5])])

    # get the clusters with the lowest distance
    shortest_distance = np.amin(np.array(cluster_distances)[:,1])
    return list(filter(lambda x: x[1] < (shortest_distance + MAX_DISTANCE), cluster_distances))

# find words in the clusters with the shortest distance to the word
def find_closest_words(word, clusters):
    word_distances = []
    for cluster_distance in clusters:
        for i in range(len(kmeans_clusters[cluster_distance[0]])):
            word_distances.append(
                [kmeans_clusters[cluster_distance[0]][i], 
                 dist.levenshtein(word, kmeans_clusters[cluster_distance[0]][i])])

    shortest_word_distance = np.amin(np.array(word_distances)[:,1].astype(np.int))
    closest_words = list(filter(lambda x: x[1] < shortest_word_distance + 2, word_distances ))
    
    return closest_words
    
# get best suggestions using combination of distance and word frequency
def get_suggestions(candidate_words):
    suggestions = []
    
    # if there is a word with distance 0, add it first, maybe it is right
    top_word = list(filter(lambda x: x[1] == 0, candidate_words))
    if len(top_word) > 0:
        word = list(top_word)[0]
        suggestions.append(word[0])
        
        
    print()
    # add all words with distance 1
    ordered_words_1 = list(filter(lambda x: x[1] == 1, candidate_words))
    if len(ordered_words_1) > 0:
        suggestions.extend(order_by_frequency(np.array(ordered_words_1)[:,0]))
    
    # add 3 words of distance 2 with the highest frequency of occurence in the text
    words_distance_2 = list(filter(lambda x: x[1] == 2, candidate_words))
    if len(words_distance_2) > 0:
        suggestions.extend(order_by_frequency(np.array(words_distance_2)[:,0])[0:3])
    
    return suggestions

# return words ordered by their occurrence frequency
def order_by_frequency(words_list):
    words_with_frequencies = list(map(lambda word: [word, word_counter[word]], words_list))
    words_with_frequencies.sort(key = lambda x: x[1], reverse = True)
    if len(words_with_frequencies) > 0:
        return np.array(words_with_frequencies)[:,0]
    else:
        return []

In [195]:
get_corrections('care')


care
are
came
case
rare
bare
cure
cart
dare
hare
card
cares
cared
fare


In [191]:
'war' in words_dictionary

True

In [ ]:
word = input()
while word != 'exit':
    print('suggestions:')
    get_corrections(word)
    print('---------------------------------')
    word = input()

In [ ]:
order_by_frequency(['the', 'car', 'happiness'])